### Import libraries and data

In [ ]:
import pandas as pd
import csv
import re
import numpy as np

In [ ]:
vols = pd.read_csv("volumes.csv")
vols = vols.drop("Unnamed: 0", axis=1)

In [ ]:
temps = pd.read_csv("temperatures.csv")
temps = temps.drop("Unnamed: 0", axis=1)

In [ ]:
unitformula = pd.read_csv("scaledformulae.csv")
unitformula = unitformula.drop("Unnamed: 0", axis=1)

### Setting models

In [ ]:
from sklearn import linear_model, svm, kernel_ridge
regVols = linear_model.LinearRegression()
regAlpha = linear_model.LinearRegression()

### Percentage function

In [ ]:
def percCalc (row):
    predictedvolume = row["sum"]
    vols = row["Volume"]
    percentage = predictedvolume / vols
    if (percentage >=0.95) and (percentage <=1.05):
        return 1
    else:
        return 0

In [ ]:
#allowedElements = {"H":0,"He":0,"Li":0,"Be":0,"B":0,"C":0,"N":0,"O":0,"F":0,"Ne":0,
#"Na":0,"Mg":0,"Al":0,"Si":0,"P":0,"S":0,"Cl":0,"Ar":0,"K":0,"Ca":0,
#"Sc":0,"Ti":0,"V":0,"Cr":0,"Mn":0,"Fe":0,"Co":0,"Ni":0,"Cu":0,"Zn":0,
#"Ga":0,"Ge":0,"As":0,"Se":0,"Br":0,"Kr":0,"Rb":0,"Sr":0,"Y":0,"Zr":0,
#"Nb":0,"Mo":0,"Tc":0,"Ru":0,"Rh":0,"Pd":0,"Ag":0,"Cd":0,"In":0,"Sn":0,
#"Sb":0,"Te":0,"I":0,"Xe":0,"Cs":0,"Ba":0,"La":0,"Ce":0,"Pr":0,"Nd":0,
#"Pm":0,"Sm":0,"Eu":0,"Gd":0,"Tb":0,"Dy":0,"Ho":0,"Er":0,"Tm":0,"Yb":0,
#"Lu":0,"Hf":0,"Ta":0,"W":0,"Re":0,"Os":0,"Ir":0,"Pt":0,"Au":0,"Hg":0,
#"Tl":0,"Pb":0,"Bi":0,"Po":0,"At":0,"Rn":0,"Fr":0,"Ra":0,"Ac":0,"Th":0,
#"Pa":0,"U":0,"Np":0,"Pu":0,"Am":0,"Cm":0,"Bk":0,"Cf":0,"Es":0,"Fm":0,
#"Md":0,"No":0,"Lr":0,"Rf":0,"Db":0,"Sg":0,"Bh":0,"Hs":0,"Mt":0}
#alldata = pd.concat([unitformula, temps, vols], axis =1)

A concatination was attempted so that when a certain entry was omitted the corresponding temperature and volume values could also easily be omitted.

### Loop algorithm

In [ ]:
alpha = 0.00
x2=pd.DataFrame()
keeptemps = temps
keepvols = vols
elements = unitformula.columns.values
unitformula["Keep"] = 1
dropidx = 1
for step in range(1,5):
    print step, unitformula["Keep"].sum()
    regVols.fit(unitformula[unitformula.Keep == 1], keepvols.div(keeptemps.Temperature*alpha+1, axis="index"))
    w = regVols.coef_[0]
    for line in zip(elements, w):
        x2[line[0]] = unitformula[line[0]].apply(lambda x: x*line[1])
    x2["sum"] = x2.sum(axis=1)
    if dropidx == 1:
        vol2 = keepvols.div(x2["sum"], axis="index") - 1
    else:
        vol2 = keepvols.div(x2["sum"].drop(x2.index[[dropidx]]), axis="index") - 1
    regAlpha.fit(keeptemps, vol2)
    alpha = regAlpha.coef_[0][0]
    unitformula["Keep"] = pd.concat([x2,vols], axis=1).apply(percCalc, axis=1)
    print "Element average volume (cubic angstroms)", zip(elements,w)
    print "Alpha Value: ", alpha
    dropidx = unitformula['Keep'][unitformula["Keep"] != 1].index.tolist()
    keeptemps = temps.drop(temps.index[[dropidx]])
    keepvols = vols.drop(vols.index[[dropidx]])

Instead of concatination it was decided that the indexes of the entries that were filtered out were to be used.

Here we see an attempt at appying this percentage filtering function by using only the entries that are in range.
A list of indexes of the entries that return 0 is made; these are the entries which should be dropped for the next iteration.
This list is used to drop the corresponding temperature and volumes, to prevent mismatch and error.
An if/else statement is made to allow the initial iteration to pass using all entries

In [ ]:
print w

Printing the average volumes to check.